# Data Processing

## Main

In [42]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

pd.set_option('display.max_columns', None)


path= os.path.abspath(glob.glob("**")[1]).replace("\\", '/' ) + "/" #! Path definition 
file_aero = 'Aerocivil.txt' #! File definition 
file_price = 'Precios.txt'

## Quantities

### Data Cleaning

In [62]:
# Processing of quantities table
p_f =  path + file_aero
data = pd.read_csv(p_f, sep="\t")
df =data
df = df.replace('(en blanco)', '0') 

df =df[['Año','Mes', 'CiudadDestino','Destino','Destino_MC',
        'NombreAeropuertoDestino','CiudadOrigen','Origen','Origen_MC', 
        'NombreAeropuertoOrigen','Leg', 'MarketLeg','Airline_Cod',
        'NombredeEmpresa','SiglaEmpresa','Trafico','TipoVuelo','Ruta_Viva',
        'NumerodeVuelos','PasajerosABordo','PasajerosTransito',
        'SillasOfrecidas','KM','LF','PDEWS','Weekly_Frequencies']]

df =df.sort_values(['Año','Mes','MarketLeg']).reset_index()

df[['KM',	'LF',	'PDEWS','Weekly_Frequencies']]=df[['KM',	'LF',	'PDEWS',
                                        'Weekly_Frequencies']].astype('string')


for x in ['KM',	'LF',	'PDEWS','Weekly_Frequencies']: 
  df[x]=df[x].apply(lambda x: x.replace(',', '.'))

  
df[['NumerodeVuelos',	'PasajerosABordo',	'PasajerosTransito',	
    'SillasOfrecidas','KM',	'LF',
    'PDEWS','Weekly_Frequencies']]=df[['NumerodeVuelos',	'PasajerosABordo',	
                                      'PasajerosTransito',	'SillasOfrecidas',
                                      'KM',	'LF','PDEWS',
                                      'Weekly_Frequencies']].astype('float')

df['Date'] = df['Mes'].astype('string')  + df['Año'].astype('string')
df['Date'] = pd.to_datetime(df['Date'], format='%m%Y')

# Creating data groups

df['Company'] = df['Airline_Cod'].apply(lambda x: 'Other' if x != 'AV'and
                                        x != 'LA' and
                                        x != 'VH' and
                                        x != 'VE'
                                      else ('Avianca' if x == 'AV' 
                                          else ('Latam' if x == 'LA' 
                                            else ('Viva' if x == 'VH' 
                                              else('Easy Fly' if x=='VE' 
                                                else 'no match')))))


df['Leg'] = df['Leg'].str.replace('EOH', 'MDE')
df['MarketLeg'] = df['MarketLeg'].str.replace('EOH', 'MDE')
df['Origen'] = df['Origen'].str.replace('EOH', 'MDE')
df['Destino'] = df['Destino'].str.replace('EOH', 'MDE')
df['CiudadDestino'] =df['CiudadDestino'].str.replace('RIONEGRO - ANTIOQUIA',
                                                    'MEDELLIN')
df['CiudadOrigen'] =df['CiudadOrigen'].str.replace('MEDELLIN', 
                                                  'RIONEGRO - ANTIOQUIA')
df['Ruta_Viva'] = (df['Ruta_Viva'].str.replace('Si', '1')).astype('float')


C:\Users\navas.LAPTOP-5SHC5PM9\AppData\Local\Temp\ipykernel_23624\4154963715.py:3: DtypeWarning: Columns (3,4,5,18,24,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(p_f, sep="\t")


### Data Sampling

In [63]:
# Creating df and df_full
# df is defined as a regular and national flight after 2021
df= df[(df['TipoVuelo'] == "R") & (df['Trafico'] == "N")  & (df['Año']>= 2021)].reset_index()
# Dropping Level_0
df.drop(['level_0'], axis=1, inplace = True)

# df_full is defined as a regular and national flight at any time in the observation data (from 2011-2022)
df_full = df[(df['TipoVuelo'] == "R") & (df['Trafico'] == "N")].reset_index()

# Changing the airline code of Ultra Air
df['Airline_Cod']=df['Airline_Cod'].replace('ULS', 'OL')

# Deleting airlines that are not listed in the pricing data
df = df[(df['Airline_Cod']!= 'CM') & (df['Airline_Cod']!= 'OTROS') ]

# We grouped the data to delete duplicates

df=df.groupby(['Año','Mes','Date', 'CiudadDestino','Destino','Destino_MC',
            'CiudadOrigen','Origen','Origen_MC','Leg' , 'MarketLeg',
            'Airline_Cod','Company','Trafico','TipoVuelo'] ,
            as_index=False).agg({'Ruta_Viva':'sum','NumerodeVuelos':'sum', 
                                'NumerodeVuelos':'sum',
                                'PasajerosABordo':'sum',
                                'PasajerosTransito':'sum',
                                'SillasOfrecidas':'sum',
                                'KM':'mean','LF':'mean',
                                'PDEWS':'mean','Weekly_Frequencies':'mean' })
    

df_full=df_full.groupby(['Año','Mes','Date', 'CiudadDestino','Destino',
                        'Destino_MC','CiudadOrigen','Origen','Origen_MC',
                        'Leg', 'MarketLeg','Airline_Cod','Company' ,
                        'Trafico','TipoVuelo'] ,
                        as_index=False).agg({'Ruta_Viva':'sum',
                                            'NumerodeVuelos':'sum', 
                                            'NumerodeVuelos':'sum',
                                            'PasajerosABordo':'sum',
                                            'PasajerosTransito':'sum',
                                            'SillasOfrecidas':'sum',
                                            'KM':'mean','LF':'mean',
                                            'PDEWS':'mean',
                                            'Weekly_Frequencies':'mean' })
# we reduce the sample to one-year observation level
df['Date'] = pd.to_datetime(df['Date']).dt.to_period('M')
df = df[df['Date']>= '2021-02']



### New Variables and measures

In [65]:

# We create the HH index for passengers on board by route and period
mk_share_for_hhi = df.groupby(['Date','Leg','Airline_Cod'], 
                              as_index=False)['PasajerosABordo'].agg('sum')
                              
mk_share_for_hhi['id_HHI_treatment'] = mk_share_for_hhi['Date'].astype(str) + mk_share_for_hhi['Leg']
mk_share_for_hhi['id_m_share'] = mk_share_for_hhi['Date'].astype(str) + mk_share_for_hhi['Leg'] + mk_share_for_hhi['Airline_Cod']

consol_hhi =df.groupby(['Date', 'Leg'],
                      as_index=False)['PasajerosABordo'].agg('sum')

consol_hhi.rename(columns = {'PasajerosABordo':'PasajerosABordo_sum'}, 
                  inplace = True)

consol_hhi['id_HHI_treatment'] = consol_hhi['Date'].astype(str) + consol_hhi['Leg']
mk_share_for_hhi = mk_share_for_hhi.merge(consol_hhi, on='id_HHI_treatment', 
                                          how='left')

# We create the Market Share for passengers on board by airline, route and period
mk_share_for_hhi['m_share'] = (mk_share_for_hhi['PasajerosABordo']/ mk_share_for_hhi['PasajerosABordo_sum'])
df['id_m_share'] = df['Date'].astype(str) + df['Leg'] +df['Airline_Cod']
df_m_share = mk_share_for_hhi[['id_m_share', 'm_share']]
df = df.merge(df_m_share, on='id_m_share', how='left')



mk_share_for_hhi['HHI'] = (mk_share_for_hhi['m_share'].pow(2,axis=0))

mk_share_for_hhi = mk_share_for_hhi.groupby(['id_HHI_treatment','Date_x', 'Leg_x'], as_index=False)['HHI'].agg('sum')

df['id_HHI_treatment'] = df['Date'].astype(str) + df['Leg']
mk_share_for_hhi = mk_share_for_hhi[['id_HHI_treatment', 'HHI']]
df = df.merge(mk_share_for_hhi, on='id_HHI_treatment', how='left')


# We create the future treatment and pre-post variables by  route and period
df['OL_Presence'] =np.where(df['Airline_Cod'] == 'OL', 1, 0)
df['OL_entrance'] =np.where(df['Date'] > '2022-03', 1, 0)


treatment = df.groupby(['Date','Leg'], 
                      as_index=False)['OL_Presence','OL_entrance'].agg('sum')
TREATMENT = ['OL_Presence','OL_entrance']

for i in TREATMENT:
  treatment[i] = np.where(treatment[i]>=1,1,0)

df = df.drop(columns=['OL_Presence','OL_entrance'])


treatment_2 = treatment.groupby('Leg', as_index = False)['OL_Presence'].agg(sum)
TREATMENT_2 = ['OL_Presence']


for i in TREATMENT_2:
  treatment_2[i] = np.where(treatment_2[i]>=1,1,0)
  

treatment = treatment[[ 'OL_entrance', 'Leg', 'Date']]
treatment = treatment.merge(treatment_2, on='Leg', how='left')

treatment['id_HHI_treatment'] = treatment['Date'].astype(str) + treatment['Leg']
treatment = treatment[['id_HHI_treatment', 'OL_Presence','OL_entrance']]

df = df.merge(treatment, on='id_HHI_treatment', how='left')


C:\Users\navas.LAPTOP-5SHC5PM9\AppData\Local\Temp\ipykernel_23624\3825487084.py:40: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  treatment = df.groupby(['Date','Leg'],


## Prices

### Data Cleaning and sampling

In [66]:
#  Processing of prices table
p_f_p =  path + file_price
df_p = pd.read_csv(p_f_p, sep="\t")

df_p[['fullprice',	'Netprice', 'tax']]=df_p[['fullprice',	'Netprice', 'tax']].astype('string')

for x in ['fullprice',	'Netprice', 'tax']: 
  df_p[x]=df_p[x].apply(lambda x: x.replace(',', '.'))

df_p[['fullprice',	'Netprice', 'tax']]=round(df_p[['fullprice',	'Netprice', 'tax']].astype('float'),2)

df_p['ObservationDate'] = pd.to_datetime(df_p['ObservationDate'], format='%Y-%m-%d')
df_p['Depdate'] = pd.to_datetime(df_p['Depdate'], format='%Y-%m-%d')

df_p['Año'], df_p['Mes'], df_p['Dia'] = pd.to_datetime(df_p['Depdate']).dt.year, pd.to_datetime(df_p['Depdate']).dt.month, pd.to_datetime(df_p['Depdate']).dt.day
df_p['Month-Year'] =  pd.to_datetime(df_p['Depdate']).dt.to_period('M')

df_p['Leg'] = df_p['Leg'].str.replace('EOH', 'MDE')
df_p =df_p.sort_values(['Año','Mes','Leg']).reset_index()

# We sampling data from 2021-08 to 2022-08
df_p = df_p[(df_p['Month-Year']<='2022-08')]


### New Variables and Data grouping

In [67]:
# We create a new variable that indicates the number of days for the flight departure. 
df_p['Days for depature'] = ( df_p['ObservationDate']-df_p['Depdate'] )
df_p =df_p.sort_values(['Depdate','Days for depature'])
df_p['Days for depature']= df_p['Days for depature'].dt.days

# We grouped by month 
df_p = df_p.groupby(['Año','Mes','Month-Year','Carrier','Leg','Days for depature'], as_index=False)['Netprice','fullprice','tax'].agg('mean')

C:\Users\navas.LAPTOP-5SHC5PM9\AppData\Local\Temp\ipykernel_23624\849820958.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_p = df_p.groupby(['Año','Mes','Month-Year','Carrier','Leg','Days for depature'], as_index=False)['Netprice','fullprice','tax'].agg('mean')


## Joining quantities and prices

In [68]:
# We define new data frames
df_APP1 = df
df_p_month_day_APP1 = df_p


In [69]:
# We pivot the price data
df_p_month_day_pivot_APP1 = df_p_month_day_APP1.pivot( index =[ 'Año','Mes','Month-Year','Carrier',"Leg"], columns=['Days for depature'], values=['fullprice'])
df_p_month_day_pivot_APP1.columns = df_p_month_day_pivot_APP1.columns.droplevel(0) 
df_p_month_day_pivot_APP1.columns.name = None
df_p_month_day_pivot_APP1 = df_p_month_day_pivot_APP1.reset_index()

# We create an ID code (year + month + airline + route ) 
df_p_month_day_pivot_APP1['id'] = df_p_month_day_pivot_APP1['Año'].astype(str) + df_p_month_day_pivot_APP1['Mes'].astype(str) + df_p_month_day_pivot_APP1['Carrier'] +df_p_month_day_pivot_APP1['Leg']
df_APP1['id'] = df_APP1['Año'].astype(str) + df_APP1['Mes'].astype(str) + df_APP1['Airline_Cod'] +df_APP1['Leg']

# We condition the presence of data in our samples in order to develop the join exercise further.
df_APP1 = df_APP1[df_APP1['Leg'].isin(list(df_p.groupby('Leg', as_index = False).count()['Leg']))]
df_p_month_day_pivot_APP1 = df_p_month_day_pivot_APP1[df_p_month_day_pivot_APP1['id'].isin(list(df_APP1.groupby('id', as_index = False).count()['id']))]
df_APP1 = df_APP1[df_APP1['id'].isin(list(df_p_month_day_pivot_APP1.groupby('id', as_index = False).count()['id']))]

# We drop some columns that we do not use
df_p_month_day_pivot_APP1 = df_p_month_day_pivot_APP1.drop(columns=['Año','Mes','Month-Year','Carrier','Leg'])

# We join the Data
full_data_APP1 = pd.merge(df_APP1,df_p_month_day_pivot_APP1, on=["id","id"])
full_data_APP1= full_data_APP1.sort_values(['Date','Leg'])




### Data processing verification and export 




In [100]:

os.path.abspath(glob.glob('*/')[0]).replace("\\", '/' ).replace("/Data", '/Data_for_regresions_APP_1.xlsx' )

'c:/Users/navas.LAPTOP-5SHC5PM9/OneDrive - Universidad EAFIT/GitHub/Colombian-Air-Market-Data-Processing/Data_for_regresions_APP_1.xlsx'

In [ ]:

if full_data_APP1['id'].nunique()/ len(full_data_APP1)==1 :

  # To excel
  full_data_APP1.to_excel(os.path.abspath(glob.glob('*/')[0]).replace("\\", '/' ).replace("/Data", '/Data_for_regresions_APP_1.xlsx' ))
  # To stata
  full_data_APP1_to_stata = full_data_APP1
  full_data_APP1_to_stata['Date'] = full_data_APP1_to_stata['Año'].astype(str)+'-' +full_data_APP1_to_stata['Mes'].astype(str)
  full_data_APP1_to_stata.to_stata(os.path.abspath(glob.glob('*/')[0]).replace("\\", '/' ).replace("/Data", '/Data_for_regresions_APP_1.dta' ))

  print('The join exercise was successfully developed. We do not have duplicates')

else:
  print("We need to review de data processing. An error was made")

# STATA CODES AND PROCESSING

In [104]:
import stata_setup
#For windows 
stata_setup.config('C:/Program Files/Stata17/', 'mp')

# For linux 'uncomment' the following line
#stata_setup.config("/usr/local/stata17/", "mp")


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Unlimited-user 2-core network, expiring  6 Oct 2023
Serial number: 501709313513
  Licensed to: SANTIAGO NAVAS
               EAFIT

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


In [108]:
%%stata


c:\Users\navas.LAPTOP-5SHC5PM9\OneDrive - Universidad EAFIT\GitHub\Colombian-Ai
> r-Market-Data-Processing
